In [1]:
from slop_gen.generators.story_gen.planning import (
    Parameters,
    PostProcessing,
    generate_high_level_plan,
)
from slop_gen.generators.story_gen.scene_gen import (
    generate_all_scenes,
    SceneDescription,
)
from slop_gen.generators.story_gen.images import generate_images_for_scenes
import os

/Users/neelin2/anaconda3/envs/slop_gen/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Input

In [2]:
story: str = """
Depression is quiet.

Depression isn’t words, it’s the words we don’t say.

It’s telling everyone we’re fine after another night of tossing and turning.

It’s in the fake smile we give our mothers because truly, we can’t be the ones to break her heart.

It’s the questionable look from a friend.

The small push from your dog against your leg, wondering why you’ve been so distant.

It’s in the grayish colors of my walls. The red used to burn so bright but like any other flame, they’ve dimmed.

It’s in the lonely eyes of my sister. In the raging glare do my brother; because for some reason, he can’t do anything but fight anymore.

It’s the lump in our chests we can’t get rid of. It’s sticks to us, and weighs like wet cement. With every step we feel it shift from side to side in our hearts, swinging us off balance.

It’s in the soft, broken eyes of my boyfriend. Who’s smile is beginning to wear.

Depression is in the world around me. It’s in the endless fighting, the killing, the shooting, the looting, the burning.

It’s in every day that burns like a thousand suns. It’s in the thick frosts of winter.

It’s in my best friend, and the bruises her father gave to her like roses from a garden.

It’s in my Father, and the bottles that collect by his bed side. It’s in every cigarette, in every dismissive shake of his head.

It’s in me. Inside my thoughts, in every crevice of my broken and shattered heart. It’s in my words, tangling around my numb tongue crying to escape.

It’s in my bed, chaining me to the spring-ridden mattress.

It’s in my bathroom, in the looming reflection of my mirror.

Depression is quiet, until it’s not.

Depression is simple, until it’s not.

Depression is in anything and everything. . . Until it’s not.
"""

In [3]:
BASE_IMAGE_OUTPUT_DIR = "assets/generated_images"

parameters: Parameters = {
    "story": story,
    "director_prompt": "Fantasy dark academia, oil painting style",
    "character_design": None,
    "video_gen": False,
    "music": False,
    "music_file": None,
    "post_processing": [PostProcessing.PAN, PostProcessing.CAPTION],
    "high_level_plan": None,
    "scene_descriptions": None,
    "image_paths": None,
}

# Markdown

In [4]:
parameters["high_level_plan"] = generate_high_level_plan(parameters)
print(parameters["high_level_plan"])


INFO:httpx:HTTP Request: POST https://api.ai.it.cornell.edu/v1/chat/completions "HTTP/1.1 200 OK"


# Visual Style
The video will embrace a "fantasy dark academia" aesthetic, rendered in a painterly, oil painting style. The palette will be dominated by muted grays, deep reds, and shadowy blues, with occasional bursts of color to signify emotional intensity. Lighting will be soft and diffused, with chiaroscuro effects to evoke depth and introspection. Textures will be rich and tactile, with visible brushstrokes and layered shadows, creating a sense of timeless melancholy and poetic realism. The world will feel both grounded and slightly surreal, as if the characters inhabit a dreamlike, scholarly realm where emotion and environment are deeply intertwined.

# Visual Flow
The video will unfold as a series of interconnected vignettes, each seamlessly transitioning into the next through visual motifs (e.g., a lingering hand, a shifting shadow, a recurring object like a book or a wilted rose). The camera will move slowly and deliberately, often lingering on small, telling details—a trembli

# Generate Scenes

In [5]:
NUM_SCENES_PER_ITERATION = 3  # Define how many scenes to generate per call to the underlying iterative function
MAX_ITERATIONS = 15  # Define a maximum number of iterations for the wrapper function

if parameters["high_level_plan"] is not None:
    print(f"Generating all scenes for the story...")
    try:
        all_generated_scenes = generate_all_scenes(
            story=parameters["story"],
            high_level_plan=parameters["high_level_plan"],
            num_scenes_per_iteration=NUM_SCENES_PER_ITERATION,
            max_iterations=MAX_ITERATIONS,
        )
        parameters["scene_descriptions"] = [
            scene.model_dump() for scene in all_generated_scenes
        ]  # Store as list of dicts

        print("\nGenerated Scenes:")
        if parameters["scene_descriptions"]:
            for i, scene in enumerate(parameters["scene_descriptions"]):
                print(f"  Scene {i+1}:")
                print(f"    Text: {scene['text']}")
                print(f"    Description: {scene['description']}")
        else:
            print("  No scenes were generated in this batch.")

    except Exception as e:
        print(f"Error during scene generation: {e}")
else:
    raise ValueError("High-level plan is missing.")

Generating all scenes for the story...
Scene generation iteration 1...


INFO:httpx:HTTP Request: POST https://api.ai.it.cornell.edu/v1/chat/completions "HTTP/1.1 200 OK"


Scene generation iteration 2...


INFO:httpx:HTTP Request: POST https://api.ai.it.cornell.edu/v1/chat/completions "HTTP/1.1 200 OK"


Scene generation iteration 3...


...

Scene generation iteration 5...


INFO:httpx:HTTP Request: POST https://api.ai.it.cornell.edu/v1/chat/completions "HTTP/1.1 200 OK"


Scene generation iteration 6...


INFO:httpx:HTTP Request: POST https://api.ai.it.cornell.edu/v1/chat/completions "HTTP/1.1 200 OK"


'DONE' signal received. Story fully processed.

Generated Scenes:
  Scene 1:
    Text: It’s the questionable look from a friend.
    Description: A close-up portrait of a friend in a painterly, oil painting style with visible brushstrokes, embodying the fantasy dark academia aesthetic. The friend’s face is half in shadow, eyes sharp and searching, conveying concern and unspoken questions. The background is a blurred swirl of muted grays and shadowy blues, with hints of deep red, creating a moody, introspective atmosphere. The lighting is soft and diffused, with chiaroscuro effects highlighting the contours of the face. The friend wears layered, slightly disheveled academic clothing in muted tones, fitting the dark academia style. The overall mood is one of quiet tension and emotional depth.
  Scene 2:
    Text: The small push from your dog against your leg, wondering why you’ve been so distant.
    Description: An intimate scene rendered in a painterly oil painting style with rich text

# Image Generation

In [6]:
if parameters["scene_descriptions"]:
    print(
        f"\nStarting image generation for {len(parameters['scene_descriptions'])} scenes..."
    )
    # Ensure the base output directory exists
    if not os.path.exists(BASE_IMAGE_OUTPUT_DIR):
        os.makedirs(BASE_IMAGE_OUTPUT_DIR)
        print(f"Created image output directory: {BASE_IMAGE_OUTPUT_DIR}")

    parameters["image_paths"] = generate_images_for_scenes(
        scene_descriptions=parameters["scene_descriptions"],
        base_output_dir=BASE_IMAGE_OUTPUT_DIR,
    )
    if parameters["image_paths"]:
        print(f"\nSuccessfully generated {len(parameters['image_paths'])} images:")
        for path in parameters["image_paths"]:
            print(f"  - {path}")
    else:
        print("\nNo images were generated.")
else:
    print("\nSkipping image generation as no scene descriptions are available.")

INFO:slop_gen.generators.story_gen.images:Ensured base output directory exists: assets/generated_images
INFO:slop_gen.generators.story_gen.images:Processing scene 1/16: Generating image for 'A close-up portrait of a friend in a painterly, oi...' -> assets/generated_images/scene_0.png
INFO:slop_gen.generators.story_gen.images:Generating image for prompt: 'A close-up portrait of a friend in a painterly, oi...' with size 1024x1536



Starting image generation for 16 scenes...
Created image output directory: assets/generated_images


ERROR:slop_gen.generators.story_gen.images:Error generating or saving image for prompt 'A close-up portrait of a friend in a painterly, oil painting style with visible brushstrokes, embodying the fantasy dark academia aesthetic. The friend’s face is half in shadow, eyes sharp and searching, conveying concern and unspoken questions. The background is a blurred swirl of muted grays and shadowy blues, with hints of deep red, creating a moody, introspective atmosphere. The lighting is soft and diffused, with chiaroscuro effects highlighting the contours of the face. The friend wears layered, slightly disheveled academic clothing in muted tones, fitting the dark academia style. The overall mood is one of quiet tension and emotional depth.': 400 Client Error: Bad Request for url: https://api.ai.it.cornell.edu/v1/images/generations
ERROR:slop_gen.generators.story_gen.images:Failed to generate image for scene 0 with prompt: A close-up portrait of a friend in a painterly, oil painting style wit


No images were generated.


# Audio Generation

# Post-processing